<a href="https://colab.research.google.com/github/Ravindra1972/Anaytics-in-finance-using-Python/blob/main/Black_Scholes_pricing_and_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime as dt
from scipy import stats
from math import log, sqrt, exp
from scipy.optimize import fsolve

class european_option:

    def __init__(self, S0, K, t, M, r, sigma, d, CP, C=10):

        '''
        Attributes:
        ------------------------------------------------------------------------------------------
        S0 (initial underlying level),
        K (option strike price),
        t (pricing date),                  ## Can enter in datetime format or str ('dd/mm/yyyy') or years (float)
        M (maturity date),                 ## Same as t
        r (constant risk-free short rate), ## For 5%, enter 0.05
        sigma (volatility),                ## For 5%, enter 0.05
        d (continuous dividend rate),      ## For 5%, enter 0.05
        CP (call or put),                  ## Enter 'Call'/'C' or 'Put'/'P' in any case
        C (market price of the option)     ## Optional - only used for implied vol. method (default = 10)
        ------------------------------------------------------------------------------------------

        Methods:
        ------------------------------------------------------------------------------------------
        value (return present value of the option),
        imp_vol (implied volatility given market price),
        delta (option delta),
        gamma (option gamma),
        vega (option vega),
        theta (option theta),
        rho (option rho)
        ------------------------------------------------------------------------------------------
        '''

        self.S0 = S0
        self.K = K
        self.t = t
        self.M = M
        self.r = r
        self.sigma = sigma
        self.d = d
        self.CP = CP
        self.C = C
        self.refresh()

    def refresh(self):

        if type(self.t).__name__ == 'str':
            self.t= dt.strptime(self.t, '%m/%d/%Y')
        if type(self.M).__name__ == 'str':
            self.M = dt.strptime(self.M, '%m/%d/%Y')
        if self.CP.lower() in ['call', 'c']:
            self.CP = 'call'
        elif self.CP.lower() in ['put', 'p']:
            self.CP = 'put'
        else:
            raise ValueError("Check value of variable CP - Call/C or Put/P allowed!")
        if self.t > self.M:
            raise ValueError("Pricing date later than maturity!")

        if type(self.t).__name__ in ['int', 'float']:
            self.T = self.M - self.t
        else:
            self.T = (self.M - self.t).days/365.0


    def d1_d2(self):

        self.d1 = (log(self.S0 / self.K) + (self.r - self.d + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * sqrt(self.T))
        self.d2 = self.d1 - self.sigma * sqrt(self.T)

    def value(self):

        self.refresh()
        self.d1_d2()

        if self.CP == 'call':
            value = (self.S0 * exp(-self.d * self.T) * stats.norm.cdf(self.d1, 0.0, 1.0) - self.K * exp(-self.r * self.T) * stats.norm.cdf(self.d2, 0.0, 1.0))
        else:
            value = (self.K * exp(-self.r * self.T) * stats.norm.cdf(-self.d2, 0.0, 1.0) - self.S0 * exp(-self.d * self.T) * stats.norm.cdf(-self.d1, 0.0, 1.0))
        return value

    def imp_vol(self):

        self.refresh()
        self.d1_d2()

        option = european_option(self.S0, self.K, self.t, self.M, self.r, self.sigma, self.d, self.CP, self.C)
        option.refresh()
        option.d1_d2()

        def difference(sig):
            option.sigma = sig
            return option.value() - option.C
        iv = fsolve(difference, option.sigma)[0]
        return iv

    def delta(self):

        self.refresh()
        self.d1_d2()

        if self.CP == 'call':
            delta = exp(-self.d * self.T) * stats.norm.cdf(self.d1, 0.0, 1.0)
        else:
            delta = exp(-self.d * self.T) * (stats.norm.cdf(self.d1, 0.0, 1.0) - 1)
        return delta

    def gamma(self):

        self.refresh()
        self.d1_d2()

        gamma = (exp(-self.d * self.T) * stats.norm.pdf(self.d1, 0.0, 1.0)) / (self.S0 * self.sigma * sqrt(self.T))
        return gamma

    def vega(self):

        self.refresh()
        self.d1_d2()

        vega = self.S0 * exp(-self.d * self.T) * stats.norm.pdf(self.d1, 0.0, 1.0) * sqrt(self.T)
        return vega

    def theta(self):

        self.refresh()
        self.d1_d2()

        if self.CP == 'call':
            theta = ( -(self.S0 * exp(-self.d * self.T) * stats.norm.pdf(self.d1, 0.0, 1.0) * self.sigma / (2 * sqrt(self.T)))
                        - (self.r * self.K * exp(-self.r * self.T) * stats.norm.cdf(self.d2, 0.0, 1.0))
                        + (self.d * self.S0 * exp(-self.d * self.T) * stats.norm.cdf(self.d1, 0.0, 1.0)))
        else:
            theta = ( -(self.S0 * exp(-self.d * self.T) * stats.norm.pdf(self.d1, 0.0, 1.0) * self.sigma / (2 * sqrt(self.T)))
                        + (self.r * self.K * exp(-self.r * self.T) * stats.norm.cdf(-self.d2, 0.0, 1.0))
                        - (self.d * self.S0 * exp(-self.d * self.T) * stats.norm.cdf(-self.d1, 0.0, 1.0)))
        return theta

    def rho(self):

        self.refresh()
        self.d1_d2()

        if self.CP == 'call':
            rho = self.K * self.T * exp(-self.r * self.T) * stats.norm.cdf(self.d2, 0.0, 1.0)
        else:
            rho = - self.K * self.T * exp(-self.r * self.T) * stats.norm.cdf(-self.d2, 0.0, 1.0)
        return rho

In [4]:
# Cox-Ross-Rubinstein, Jarrow Rudd and Tian's Binomial Model
# European Option Valuation

import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import quad
mpl.rcParams['font.family'] = 'serif'

# Model Parameters
#
S0 = 100.0  # index level
K = 100.0  # option strike
T = 1.0  # maturity date
r = 0.05  # risk-less short rate
sigma = 0.2  # volatility


# Valuation Function
def CRR_option_value(S0, K, T, r, sigma, otype, M=4):
    ''' Cox-Ross-Rubinstein European option valuation.

    Parameters
    ==========
    S0 : float
        stock/index level at time 0
    K : float
        strike price
    T : float
        date of maturity
    r : float
        constant, risk-less short rate
    sigma : float
        volatility
    otype : string
        either 'call' or 'put'
    M : int
        number of time intervals
    '''
    # Time Parameters
    dt = T / M  # length of time interval
    df = math.exp(-r * dt)  # discount per interval

    # Binomial Parameters
    u = math.exp(sigma * math.sqrt(dt))  # up movement
    d = 1 / u  # down movement
    q = (math.exp(r * dt) - d) / (u - d)  # martingale branch probability

    # Array Initialization for Index Levels
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md

    # Inner Values
    if otype == 'call':
        V = np.maximum(S - K, 0)  # inner values for European call option
    else:
        V = np.maximum(K - S, 0)  # inner values for European put option

    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        V[0:M - z, t] = (q * V[0:M - z, t + 1] +
                         (1 - q) * V[1:M - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]



def Jarrow_Rudd_option_value(S0, K, T, r, sigma, otype, M=4):
    ''' Jarrow and Rudd's modified Cox-Ross-Rubinstein European option valuation.

    Parameters
    ==========
    S0 : float
        stock/index level at time 0
    K : float
        strike price
    T : float
        date of maturity
    r : float
        constant, risk-less short rate
    sigma : float
        volatility
    otype : string
        either 'call' or 'put'
    M : int
        number of time intervals
    '''
    # Time Parameters
    dt = T / M  # length of time interval
    df = math.exp(-r * dt)  # discount per interval

    # Binomial Parameters
    u = math.exp((r-(sigma**2)/2)*dt + sigma * math.sqrt(dt))  # up movement
    d = math.exp((r-(sigma**2)/2)*dt - sigma * math.sqrt(dt))  # down movement
    q = (math.exp(r * dt) - d) / (u - d)  # martingale branch probability

    # Array Initialization for Index Levels
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md

    # Inner Values
    if otype == 'call':
        V = np.maximum(S - K, 0)  # inner values for European call option
    else:
        V = np.maximum(K - S, 0)  # inner values for European put option

    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        V[0:M - z, t] = (q * V[0:M - z, t + 1] +
                         (1 - q) * V[1:M - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]


def Tian_option_value(S0, K, T, r, sigma, otype, M=4):
    ''' Tian's modfied Cox-Ross-Rubinstein European option valuation.

    Parameters
    ==========
    S0 : float
        stock/index level at time 0
    K : float
        strike price
    T : float
        date of maturity
    r : float
        constant, risk-less short rate
    sigma : float
        volatility
    otype : string
        either 'call' or 'put'
    M : int
        number of time intervals
    '''
    # Time Parameters
    dt = T / M  # length of time interval
    df = math.exp(-r * dt)  # discount per interval

    # Binomial Parameters
    v = math.exp((sigma**2) * dt)
    u = 0.5 * math.exp(r * dt) * v * (v + 1 + math.sqrt(v**2 + 2*v -3))  # up movement
    d = 0.5 * math.exp(r * dt) * v * (v + 1 - math.sqrt(v**2 + 2*v -3))  # down movement
    q = (math.exp(r * dt) - d) / (u - d)  # martingale branch probability

    # Array Initialization for Index Levels
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md

    # Inner Values
    if otype == 'call':
        V = np.maximum(S - K, 0)  # inner values for European call option
    else:
        V = np.maximum(K - S, 0)  # inner values for European put option

    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        V[0:M - z, t] = (q * V[0:M - z, t + 1] +
                         (1 - q) * V[1:M - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]

def plot_convergence(mmin=1, mmax=30, step_size=1):
    ''' Plots the CRR option values for increasing number of time
    intervals M against the Black-Scholes-Merton benchmark value.'''
    BSM_benchmark = BSM_call_value(S0, K, 0, T, r, sigma)
    m = range(mmin, mmax, step_size)

    CRR_values = [CRR_option_value(S0, K, T, r, sigma, 'call', M) for M in m]
    JR_values = [Jarrow_Rudd_option_value(S0, K, T, r, sigma, 'call', M) for M in m]
    Tian_values = [Tian_option_value(S0, K, T, r, sigma, 'call', M) for M in m]

    plt.figure(figsize=(9, 5))
    plt.plot(m, CRR_values, label='CRR values')
    plt.plot(m, JR_values, label='JR values')
    plt.plot(m, Tian_values, label='Tian values')
    plt.axhline(BSM_benchmark, color='r', ls='dashed', lw=1.5,
                label='BSM benchmark')
    plt.xlabel('# of binomial steps $M$')
    plt.ylabel('European call option value')
    plt.legend(loc=4)
    plt.xlim(0, mmax)


### Helper functions ###

def dN(x):
    ''' Probability density function of standard normal random variable x. '''
    return math.exp(-0.5 * x ** 2) / math.sqrt(2 * math.pi)


def N(d):
    ''' Cumulative density function of standard normal random variable x. '''
    return quad(lambda x: dN(x), -20, d, limit=50)[0]


def d1f(St, K, t, T, r, sigma):
    ''' Black-Scholes-Merton d1 function.
        Parameters see e.g. BSM_call_value function. '''
    d1 = (math.log(St / K) + (r + 0.5 * sigma ** 2)
          * (T - t)) / (sigma * math.sqrt(T - t))
    return d1


def BSM_call_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European call option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    call_value : float
        European call present value at t
    '''
    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    call_value = St * N(d1) - math.exp(-r * (T - t)) * K * N(d2)
    return call_value


def BSM_put_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European put option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    put_value : float
        European put present value at t
    '''
    put_value = BSM_call_value(St, K, t, T, r, sigma) \
        - St + math.exp(-r * (T - t)) * K
    return put_value